# Run experiments with py_experimenter
This involves specifying different active learning pipelines and evaluating them on different active learning problems. The results are then stored in a database. To specify the exact experiments and parameters to be run, the file config/exp_config.yml is used as well as parameters are filled in the function experimenter.fill_table_from_combination() below. In our case, the results will be filled into ALPBenchmark.db

For further information we refer to the docs https://tornede.github.io/py_experimenter/

In [1]:
import numpy as np
from py_experimenter.exceptions import DatabaseConnectionError
from py_experimenter.experimenter import PyExperimenter, ResultProcessor
import types
from alpbench.benchmark.BenchmarkConnector import DataFileBenchmarkConnector
from alpbench.benchmark.BenchmarkSuite import TabZillaBenchmarkSuite
from alpbench.evaluation.experimenter.DefaultSetup import ensure_default_setup
from alpbench.evaluation.experimenter.LogTableObserver import LogTableObserver, SparseLogTableObserver
from alpbench.pipeline.ActiveLearningPipeline import ActiveLearningPipeline
from alpbench.pipeline.Oracle import Oracle
import sqlite3
import pandas as pd
import os

### Get ids of the **tabzilla benchmark suite**

In [6]:
tabzilla = TabZillaBenchmarkSuite()
tabzilla_ids = tabzilla.get_openml_dataset_ids()[:2]

### Setup experiment runner


In [7]:
# loads parameters from the grid and runs each active learning pipeline on every active learning problem
class ExperimentRunner:

    def __init__(self):
        pass

    def run_experiment(self, parameters: dict, result_processor: ResultProcessor, custom_config: dict):

        dbbc = DataFileBenchmarkConnector()

        connector: DataFileBenchmarkConnector = dbbc

        OPENML_ID = int(parameters["openml_id"])
        SETTING_NAME = parameters["setting_name"]
        TEST_SPLIT_SEED = int(parameters["test_split_seed"])
        TRAIN_SPLIT_SEED = int(parameters["train_split_seed"])
        SEED = int(parameters["seed"])

        setting = connector.load_setting_by_name(SETTING_NAME)
        scenario = connector.load_or_create_scenario(
            openml_id=OPENML_ID,
            test_split_seed=TEST_SPLIT_SEED,
            train_split_seed=TRAIN_SPLIT_SEED,
            seed=SEED,
            setting_id=setting.get_setting_id(),
        )

        X_l, y_l, X_u, y_u, X_test, y_test = scenario.get_data_split()

        QUERY_STRATEGY = connector.load_query_strategy_by_name(parameters["query_strategy_name"])

        print("param learner", parameters["learner_name"])

        LEARNER = connector.load_learner_by_name(parameters["learner_name"])

        OBSERVER = [SparseLogTableObserver(result_processor, X_test, y_test)]

        ALP = ActiveLearningPipeline(
            learner=LEARNER,
            query_strategy=QUERY_STRATEGY,
            observer_list=OBSERVER,
            num_iterations=setting.get_number_of_iterations(),
            num_queries_per_iteration=setting.get_number_of_queries(),
        )

        oracle = Oracle(X_u, y_u)
        ALP.active_fit(X_l, y_l, X_u, oracle)

### Run default setup

In [8]:
# choose learning algorithms, query strategies and parameters **(chosen algorithms and their parameters
# are saved in alpbench/ in .json files**, to not make these experiments run for too long, we restrict
# ourselves to the first **4 dataset ids, 1 setting, 5 seeds, 2 learning algorithms and 3 query strategies**


def run(run_setup=False, reset_experiments=False):

    exp_config_file = "config/exp_config.yml"

    experimenter = PyExperimenter(experiment_configuration_file_path=exp_config_file)

    if run_setup:
        benchmark_connector = DataFileBenchmarkConnector()
        ensure_default_setup(dbbc=benchmark_connector)

        benchmark_connector.cleanup()

        setting_combinations = []
        setting_combinations += [{"setting_name": "small"}]

        if reset_experiments:
            experimenter.reset_experiments("running", "failed")

        else:
            experimenter.fill_table_from_combination(
                parameters={
                    "learner_name": ["rf_entropy", "svm_rbf"],
                    "query_strategy_name": ["random", "margin", "cluster_margin"],
                    "test_split_seed": np.arange(1),
                    "train_split_seed": np.arange(1),
                    "seed": np.arange(5),
                    "openml_id": tabzilla_ids,
                },
                fixed_parameter_combinations=setting_combinations,
            )

    else:
        er = ExperimentRunner()
        experimenter.execute(er.run_experiment, -1)

In [9]:
run(run_setup=True, reset_experiments=False)

2024-07-10 17:15:54,071  | py-experimenter - WARNING  | No values given for keyfield setting_name
2024-07-10 17:15:54,074  | py-experimenter - WARNING  | No values given for keyfield openml_id
2024-07-10 17:15:54,075  | py-experimenter - WARNING  | No values given for keyfield learner_name
2024-07-10 17:15:54,079  | py-experimenter - WARNING  | No values given for keyfield query_strategy_name
2024-07-10 17:15:54,082  | py-experimenter - WARNING  | No values given for keyfield test_split_seed
2024-07-10 17:15:54,084  | py-experimenter - WARNING  | No values given for keyfield train_split_seed
2024-07-10 17:15:54,087  | py-experimenter - WARNING  | No values given for keyfield seed
2024-07-10 17:15:54,089  | py-experimenter - INFO     | Found 7 keyfields
2024-07-10 17:15:54,092  | py-experimenter - WARNING  | No resultfields given
2024-07-10 17:15:54,096  | py-experimenter - INFO     | Found 2 logtables
2024-07-10 17:15:54,099  | py-experimenter - INFO     | Found logtable results__accur

### Display (empty) tables

In [10]:
# Specify the path to your .db file
db_path = "ALPBenchmark.db"

# Connect to the database
conn = sqlite3.connect(db_path)

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Get the list of tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Display the contents of each table
for table_name in tables[:1]:
    table_name = table_name[0]
    print(f"Contents of table {table_name}:")
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql_query(query, conn)
    display(df)
    print("\n")

# Close the connection
conn.close()

Contents of table results:


,ID,setting_name,openml_id,learner_name,query_strategy_name,test_split_seed,train_split_seed,seed,creation_date,status,start_date,name,machine,end_date,error
0,1,small,11,rf_entropy,random,0,0,0,2024-07-10 17:15:54,created,None,None,None,None,None
1,2,small,11,svm_rbf,random,0,0,0,2024-07-10 17:15:54,created,None,None,None,None,None
2,3,small,14,rf_entropy,random,0,0,0,2024-07-10 17:15:54,created,None,None,None,None,None
3,4,small,14,svm_rbf,random,0,0,0,2024-07-10 17:15:54,created,None,None,None,None,None
4,5,small,11,rf_entropy,margin,0,0,0,2024-07-10 17:15:54,created,None,None,None,None,None
5,6,small,11,svm_rbf,margin,0,0,0,2024-07-10 17:15:54,created,None,None,None,None,None
6,7,small,14,rf_entropy,margin,0,0,0,2024-07-10 17:15:54,created,None,None,None,None,None
7,8,small,14,svm_rbf,margin,0,0,0,2024-07-10 17:15:54,created,None,None,None,None,None
8,9,small,11,rf_entropy,cluster_margin,0,0,0,2024-07-10 17:15:54,created,None,None,None,None,None
9,10,small,11,svm_rbf,cluster_margin,0,0,0,2024-07-10 17:15:54,created,None,None,None,None,None


In [11]:
run(run_setup=False, reset_experiments=False)

2024-07-10 17:16:07,282  | py-experimenter - WARNING  | No values given for keyfield setting_name
2024-07-10 17:16:07,284  | py-experimenter - WARNING  | No values given for keyfield openml_id
2024-07-10 17:16:07,286  | py-experimenter - WARNING  | No values given for keyfield learner_name
2024-07-10 17:16:07,288  | py-experimenter - WARNING  | No values given for keyfield query_strategy_name
2024-07-10 17:16:07,290  | py-experimenter - WARNING  | No values given for keyfield test_split_seed
2024-07-10 17:16:07,291  | py-experimenter - WARNING  | No values given for keyfield train_split_seed
2024-07-10 17:16:07,293  | py-experimenter - WARNING  | No values given for keyfield seed
2024-07-10 17:16:07,295  | py-experimenter - INFO     | Found 7 keyfields
2024-07-10 17:16:07,296  | py-experimenter - WARNING  | No resultfields given
2024-07-10 17:16:07,299  | py-experimenter - INFO     | Found 2 logtables
2024-07-10 17:16:07,301  | py-experimenter - INFO     | Found logtable results__accur

param learner svm_rbf


/home/valentin/test_venv3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valentin/test_venv3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valentin/test_venv3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valentin/test_venv3/lib/python3.10/site-packages/skl

param learner rf_entropy


[codecarbon INFO @ 17:16:23] Energy consumed for RAM : 0.000017 kWh. RAM Power : 11.630144119262695 W
[codecarbon INFO @ 17:16:23] Energy consumed for all GPUs : 0.000007 kWh. Total GPU Power : 5.135883792250112 W
[codecarbon INFO @ 17:16:23] Energy consumed for all CPUs : 0.000061 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:16:23] 0.000085 kWh of electricity used since the beginning.
[codecarbon WARNING @ 17:16:23] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 17:16:23] [setup] RAM Tracking...
[codecarbon INFO @ 17:16:23] [setup] GPU Tracking...
[codecarbon INFO @ 17:16:23] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:16:23] [setup] CPU Tracking...
[codecarbon WARNING @ 17:16:23] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 17:16:24] We saw that you have a 12th Gen Intel(R) Core(TM) i7-12700H but we don't know it. Please contact us.
[codecarbon INFO @ 17:16:24] CPU Model on constant consumption m

...

/home/valentin/test_venv3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valentin/test_venv3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valentin/test_venv3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valentin/test_venv3/lib/python3.10/site-packages/skl

param learner svm_rbf


/home/valentin/test_venv3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/valentin/test_venv3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[codecarbon INFO @ 17:24:55] Energy consumed for RAM : 0.000013 kWh. RAM Power : 11.630144119262695 W
[codecarbon INFO @ 17:24:55] Energy consumed for all GPUs : 0.000019 kWh. Total GPU Power : 17.81615203972794 W
[codecarbon INFO @ 17:24:55] Energy consumed for all CPUs : 0.000046 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:24:55] 0.000077 kWh of electricity us

param learner rf_entropy


[codecarbon INFO @ 17:25:08] Energy consumed for RAM : 0.000029 kWh. RAM Power : 11.630144119262695 W
[codecarbon INFO @ 17:25:09] Energy consumed for all GPUs : 0.000022 kWh. Total GPU Power : 8.564183850281381 W
[codecarbon INFO @ 17:25:09] Energy consumed for all CPUs : 0.000121 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 17:25:09] 0.000171 kWh of electricity used since the beginning.
2024-07-10 17:25:09,815  | py-experimenter - INFO     | All configured executions finished.


### Display (filled) tables

In [12]:
# Specify the path to your .db file
db_name = "ALPBenchmark"
db_path = db_name + ".db"
# db_path = "ALPBenchmark.db"
print(db_path)
# Connect to the database
conn = sqlite3.connect(db_path)

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Get the list of tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()


if not os.path.exists("DATAFRAMES/"):
    os.makedirs("DATAFRAMES/")


# Display the contents of each table
for enum, table_name in enumerate(tables):
    table_name = table_name[0]
    print(f"Contents of table {table_name}:")
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql_query(query, conn)
    df.to_csv("DATAFRAMES/" + db_name + "_" + table_name + ".csv")

    display(df)
    print("\n")

# Close the connection
conn.close()

ALPBenchmark.db
Contents of table results:


,ID,setting_name,openml_id,learner_name,query_strategy_name,test_split_seed,train_split_seed,seed,creation_date,status,start_date,name,machine,end_date,error
0,1,small,11,rf_entropy,random,0,0,0,2024-07-10 17:15:54,done,2024-07-10 17:20:42,PyExperimenter,valentin-XPS-15-9520,2024-07-10 17:20:51,None
1,2,small,11,svm_rbf,random,0,0,0,2024-07-10 17:15:54,done,2024-07-10 17:20:07,PyExperimenter,valentin-XPS-15-9520,2024-07-10 17:20:12,None
2,3,small,14,rf_entropy,random,0,0,0,2024-07-10 17:15:54,done,2024-07-10 17:17:29,PyExperimenter,valentin-XPS-15-9520,2024-07-10 17:17:42,None
3,4,small,14,svm_rbf,random,0,0,0,2024-07-10 17:15:54,done,2024-07-10 17:23:18,PyExperimenter,valentin-XPS-15-9520,2024-07-10 17:23:24,None
4,5,small,11,rf_entropy,margin,0,0,0,2024-07-10 17:15:54,done,2024-07-10 17:22:36,PyExperimenter,valentin-XPS-15-9520,2024-07-10 17:22:45,None
5,6,small,11,svm_rbf,margin,0,0,0,2024-07-10 17:15:54,done,2024-07-10 17:22:53,PyExperimenter,valentin-XPS-15-9520,2024-07-10 17:22:58,None
6,7,small,14,rf_entropy,margin,0,0,0,2024-07-10 17:15:54,done,2024-07-10 17:17:14,PyExperimenter,valentin-XPS-15-9520,2024-07-10 17:17:28,None
7,8,small,14,svm_rbf,margin,0,0,0,2024-07-10 17:15:54,done,2024-07-10 17:23:29,PyExperimenter,valentin-XPS-15-9520,2024-07-10 17:23:36,None
8,9,small,11,rf_entropy,cluster_margin,0,0,0,2024-07-10 17:15:54,done,2024-07-10 17:21:01,PyExperimenter,valentin-XPS-15-9520,2024-07-10 17:21:10,None
9,10,small,11,svm_rbf,cluster_margin,0,0,0,2024-07-10 17:15:54,done,2024-07-10 17:23:46,PyExperimenter,valentin-XPS-15-9520,2024-07-10 17:23:51,None




Contents of table sqlite_sequence:


,name,seq
0,results,60
1,results__labeling_log,60
2,results__accuracy_log,60
3,results_codecarbon,60




Contents of table results__accuracy_log:


,ID,experiment_id,timestamp,model_dict
0,1,14,2024-07-10 17:16:13,"{""0"": {""iteration"": 0, ""test_f1"": 0.78834966, ..."
1,2,13,2024-07-10 17:16:23,"{""0"": {""iteration"": 0, ""test_f1"": 0.71966326, ..."
2,3,23,2024-07-10 17:16:39,"{""0"": {""iteration"": 0, ""test_f1"": 0.53587528, ..."
3,4,56,2024-07-10 17:16:47,"{""0"": {""iteration"": 0, ""test_f1"": 0.47182191, ..."
4,5,43,2024-07-10 17:17:00,"{""0"": {""iteration"": 0, ""test_f1"": 0.49974758, ..."
5,6,16,2024-07-10 17:17:08,"{""0"": {""iteration"": 0, ""test_f1"": 0.47182191, ..."
6,7,40,2024-07-10 17:17:14,"{""0"": {""iteration"": 0, ""test_f1"": 0.47182191, ..."
7,8,7,2024-07-10 17:17:28,"{""0"": {""iteration"": 0, ""test_f1"": 0.55453711, ..."
8,9,3,2024-07-10 17:17:42,"{""0"": {""iteration"": 0, ""test_f1"": 0.50545106, ..."
9,10,32,2024-07-10 17:17:48,"{""0"": {""iteration"": 0, ""test_f1"": 0.47182191, ..."




Contents of table results__labeling_log:


,ID,experiment_id,timestamp,data_dict
0,1,14,2024-07-10 17:16:13,"{""0"": {""iteration"": 0, ""len_X_sel"": 30, ""len_X..."
1,2,13,2024-07-10 17:16:23,"{""0"": {""iteration"": 0, ""len_X_sel"": 30, ""len_X..."
2,3,23,2024-07-10 17:16:39,"{""0"": {""iteration"": 0, ""len_X_sel"": 30, ""len_X..."
3,4,56,2024-07-10 17:16:47,"{""0"": {""iteration"": 0, ""len_X_sel"": 30, ""len_X..."
4,5,43,2024-07-10 17:17:00,"{""0"": {""iteration"": 0, ""len_X_sel"": 30, ""len_X..."
5,6,16,2024-07-10 17:17:08,"{""0"": {""iteration"": 0, ""len_X_sel"": 30, ""len_X..."
6,7,40,2024-07-10 17:17:14,"{""0"": {""iteration"": 0, ""len_X_sel"": 30, ""len_X..."
7,8,7,2024-07-10 17:17:28,"{""0"": {""iteration"": 0, ""len_X_sel"": 30, ""len_X..."
8,9,3,2024-07-10 17:17:41,"{""0"": {""iteration"": 0, ""len_X_sel"": 30, ""len_X..."
9,10,32,2024-07-10 17:17:48,"{""0"": {""iteration"": 0, ""len_X_sel"": 30, ""len_X..."




Contents of table results_codecarbon:


,ID,experiment_id,codecarbon_timestamp,project_name,run_id,duration_seconds,emissions_kg,emissions_rate_kg_sec,cpu_power_watt,gpu_power_watt,...,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud,power_usage_efficiency,offline_mode
0,1,14,2024-07-10T17:16:13,codecarbon,ff457801-3775-4d05-8d41-6b359b412835,0.578546,0.000005,0.000008,42.5,21.633560,...,12th Gen Intel(R) Core(TM) i7-12700H,1.0,1 x NVIDIA GeForce RTX 3050 Ti Laptop GPU,11.5269,48.1537,31.013718,machine,N,1.0,0
1,2,13,2024-07-10T17:16:23,codecarbon,c61f384a-1a3b-439b-a723-d2a831cf1bbb,5.179862,0.000033,0.000006,42.5,5.135884,...,12th Gen Intel(R) Core(TM) i7-12700H,1.0,1 x NVIDIA GeForce RTX 3050 Ti Laptop GPU,11.5269,48.1537,31.013718,machine,N,1.0,0
2,3,23,2024-07-10T17:16:40,codecarbon,24ff03d8-f0ab-4a96-8077-61e7b611c6ef,12.621775,0.000081,0.000006,42.5,7.677184,...,12th Gen Intel(R) Core(TM) i7-12700H,1.0,1 x NVIDIA GeForce RTX 3050 Ti Laptop GPU,11.5269,48.1537,31.013718,machine,N,1.0,0
3,4,56,2024-07-10T17:16:47,codecarbon,a7aca2cc-3d29-4af6-8bb3-ac635cf4d48e,2.387102,0.000016,0.000007,42.5,10.802796,...,12th Gen Intel(R) Core(TM) i7-12700H,1.0,1 x NVIDIA GeForce RTX 3050 Ti Laptop GPU,11.5269,48.1537,31.013718,machine,N,1.0,0
4,5,43,2024-07-10T17:17:02,codecarbon,c4e47280-68ef-4cec-9b2d-5ddfe3d43cbe,10.567764,0.000066,0.000006,42.5,6.008132,...,12th Gen Intel(R) Core(TM) i7-12700H,1.0,1 x NVIDIA GeForce RTX 3050 Ti Laptop GPU,11.5269,48.1537,31.013718,machine,N,1.0,0
5,6,16,2024-07-10T17:17:08,codecarbon,0b867edc-2450-400b-840f-847dd3e8851d,1.798576,0.000012,0.000007,42.5,9.080435,...,12th Gen Intel(R) Core(TM) i7-12700H,1.0,1 x NVIDIA GeForce RTX 3050 Ti Laptop GPU,11.5269,48.1537,31.013718,machine,N,1.0,0
6,7,40,2024-07-10T17:17:14,codecarbon,1ac49f27-8992-48b7-bcc5-d5905664028d,1.993975,0.000013,0.000006,42.5,5.617279,...,12th Gen Intel(R) Core(TM) i7-12700H,1.0,1 x NVIDIA GeForce RTX 3050 Ti Laptop GPU,11.5269,48.1537,31.013718,machine,N,1.0,0
7,8,7,2024-07-10T17:17:29,codecarbon,8d025b84-da2e-4dab-9144-62ca8eb2b1e4,10.362005,0.000064,0.000006,42.5,5.347997,...,12th Gen Intel(R) Core(TM) i7-12700H,1.0,1 x NVIDIA GeForce RTX 3050 Ti Laptop GPU,11.5269,48.1537,31.013718,machine,N,1.0,0
8,9,3,2024-07-10T17:17:42,codecarbon,9a7f60ab-fbd8-4546-bbd0-a1eb8355ba5d,8.132112,0.000054,0.000007,42.5,8.528670,...,12th Gen Intel(R) Core(TM) i7-12700H,1.0,1 x NVIDIA GeForce RTX 3050 Ti Laptop GPU,11.5269,48.1537,31.013718,machine,N,1.0,0
9,10,32,2024-07-10T17:17:48,codecarbon,d70a4976-f50c-455d-bc46-789154c360d5,2.316687,0.000016,0.000007,42.5,10.568430,...,12th Gen Intel(R) Core(TM) i7-12700H,1.0,1 x NVIDIA GeForce RTX 3050 Ti Laptop GPU,11.5269,48.1537,31.013718,machine,N,1.0,0
